- ## ****Temporal features: hour/day/month/year, season, holiday flags (if available)****
  - Since we do not have a complete history (year/month) in the core Instacart dataset, we will focus on the available features (hour and day) and build “flags” from them that represent time periods and seasons.

- ## **Implemented Features**

- ****All available time data within the original files has been exhausted. The current feature-engineered data is perfectly sufficient to model user behavior based on daily and weekly patterns.****

In [ ]:
# Extracting available raw time data from the command table
# We use drop_duplicates to ensure that rows are not duplicated when merging
temporal_info = orders[['order_id', 'order_hour_of_day', 'order_dow', 'days_since_prior_order']].drop_duplicates()

In [ ]:
# Standardizing data types to ensure fast and warning-free integration.
temporal_info['order_id'] = temporal_info['order_id'].astype('int32')
My_Data['order_id'] = My_Data['order_id'].astype('int32')

In [ ]:
# Merging Time Features with the My_Data Main Table
# We only select columns that have not been merged previously to avoid duplication
cols_to_merge = temporal_info.columns.difference(My_Data.columns).tolist() + ['order_id']
My_Data = My_Data.merge(temporal_info[cols_to_merge], on='order_id', how='left')

   - ### ****Hour of Day****

In [ ]:
# Build the "Day Periods" feature based on the available on-demand clock.
def get_day_period(hour):
    if 5 <= hour < 12: return 1    # Morning
    elif 12 <= hour < 17: return 2  # Afternoon
    elif 17 <= hour < 21: return 3  # Evening
    else: return 4                 # Night

My_Data['day_period'] = My_Data['order_hour_of_day'].apply(get_day_period).astype('int8')


- ### ****Day of the Week****

In [ ]:
# Build a "Weekend Flag" feature based on the available day number
# In Instacart: Days 0 and 1 represent peak demand (often Saturday and Sunday)
My_Data['is_weekend'] = My_Data['order_dow'].apply(lambda x: 1 if x in [0, 1] else 0).astype('int8')

In [ ]:
# 6. Addressing the "Days Since Last Order" feature that is already available
My_Data['days_since_prior_order'] = My_Data['days_since_prior_order'].fillna(0).astype('float32')
# Clean the memory immediately after completion
del temporal_info
gc.collect()
print(['day_period', 'is_weekend', 'days_since_prior_order'])

['day_period', 'is_weekend', 'days_since_prior_order']
